In [1]:
from dotenv import load_dotenv
import os
from pathlib import Path
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

## Load test dataset

In [13]:
dotenv_path = "../.env"
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

gemini_api_key = os.getenv("GEMINI_API_KEY3") # API KEY for baseline.

In [14]:
test_df = pd.read_csv('../Splits/combined/sub_test.csv')
# test_df = pd.read_csv('../Splits/harder_test/sub_test.csv')
test_df

,Unnamed: 0,writer,problem_id,submission_id,website,source,dataset_type
0,0,Human,NaN,13688015.0,https://codeforces.com,#include<iostream>\n#include<cstdio>\n#include...,test
1,1,Human,NaN,36590979.0,https://codeforces.com,#include <bits/stdc++.h>\n\nusing namespace st...,test
2,2,AI,1909G,NaN,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...,test
3,3,Human,toi7_sewer,305644.0,https://programming.in.th,// YoruoniVamp - VTUBE\n// Pragma Credit to Di...,test
4,4,Human,NaN,36539175.0,https://codeforces.com,#include <bits/stdc++.h>\n// #include <ext/rop...,test
...,...,...,...,...,...,...,...
2395,2395,Human,NaN,35426753.0,https://codeforces.com,/**\n * author: skmonir\n * created: 17...,test
2396,2396,AI,1976F,NaN,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...,test
2397,2397,AI,272D,NaN,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...,test
2398,2398,Human,1112,103903.0,https://programming.in.th,#include<bits/stdc++.h>\n\n\nusing namespace s...,test


In [15]:
# Prompt URL
prompt_path = "./Prompts/PromptV1.txt"

with open(prompt_path,'r') as _prompt:
  prompt = _prompt.read()
prompt

'You are the best C++ code detector. Your job is to determine whether the competitive programming source code is AI-generated or written by a human.\n\n### Steps:\n1. **Read the source code carefully** to understand what does the code doing.\n2. **Identify the patterns of code** and analyze whter it\'s AI-generated or not, like checking unnesscary space, variables, new lines, or comments.\n3. **Determine if it\'s Human or AI** from the data you have.\n4. **From the reason** Answer if it AI or Human.\n\nYou must respond in JSON format, like this:\n```json\n{   \n    "reason": "Why this code is human-written or AI-generated"\n    "answer": "human" / "AI",  \n}\n```\n- You must not provide any other text, except JSON.\n- In the reason part, don\'t use symbol like " , ` , \\n, use only normal character.\nHere is C++ source code:\n'

In [16]:
test_df['prompt'] = prompt + "\n\n" + test_df['source']
test_df['prompt'][0]

'You are the best C++ code detector. Your job is to determine whether the competitive programming source code is AI-generated or written by a human.\n\n### Steps:\n1. **Read the source code carefully** to understand what does the code doing.\n2. **Identify the patterns of code** and analyze whter it\'s AI-generated or not, like checking unnesscary space, variables, new lines, or comments.\n3. **Determine if it\'s Human or AI** from the data you have.\n4. **From the reason** Answer if it AI or Human.\n\nYou must respond in JSON format, like this:\n```json\n{   \n    "reason": "Why this code is human-written or AI-generated"\n    "answer": "human" / "AI",  \n}\n```\n- You must not provide any other text, except JSON.\n- In the reason part, don\'t use symbol like " , ` , \\n, use only normal character.\nHere is C++ source code:\n\n\n#include<iostream>\n#include<cstdio>\n#include<cstring>\n#include<algorithm>\n#include<cmath>\nusing namespace std;\nconst int N=1111;\nstruct node\n{\n int t

In [17]:
BACKUP_PATH = "./Backups/Result-Gemini2.0-Normal.csv"
# BACKUP_PATH = "./Backups/Result-Gemini2.0-Hard.csv"

def save_responses(res):
    df = pd.DataFrame(res, columns=['back_up'])
    df.to_csv(BACKUP_PATH,index=False)

In [18]:
try:
    saved_df = pd.read_csv(BACKUP_PATH)
    answer = saved_df['back_up'].to_list()
except:
    answer = []

In [19]:
len(answer)

1300

In [20]:
from google import genai
from google.genai import types
import pathlib
import httpx
import time
from IPython.display import clear_output
client = genai.Client(api_key=gemini_api_key)
err_logs = []

err_cnt = 0

for index, row in test_df[len(answer):].iterrows():
  if index%50 == 0 and index != 0:
    save_responses(answer)
  while True:
    if err_cnt >= 10:
      break
    try:
      rprompt = row['prompt']
      response = client.models.generate_content(model="gemini-2.0-flash-lite",contents=[rprompt])
      s = response.text
      
      clear_output()
      print(f"ASKED : {index}")
      print(response.text)
      err_cnt = 0
      answer.append(s)
      time.sleep(1)
      break
    except Exception as e:
      err_cnt += 1
      err_logs.append((index,e))
      clear_output()
      print(f"ERROR at {index} : {e}")
      save_responses(answer)
      time.sleep(15)
save_responses(answer)

ASKED : 2399
```json
{
    "reason": "The code exhibits a clear logical structure uses standard library functions and attempts to find a specific pattern based on input strings which indicates a human approach. The code is well-structured and easy to read without unnecessary fluff. The use of different combinations and permutations further suggests a human-like problem-solving strategy.",
    "answer": "human"
}
```


In [21]:
save_responses(answer)

In [22]:
for idx, res in enumerate(answer):
   answer[idx] = res.text

AttributeError: 'str' object has no attribute 'text'

In [23]:
answer

['```json\n{\n    "reason": "The code demonstrates a clear understanding of the problem\'s logic with well-defined structures and conditional statements It includes comments and uses standard library functions The formatting and variable names are reasonable indicating human authorship",\n    "answer": "human"\n}\n```',
 '```json\n{\n    "reason": "The code exhibits a structured approach with clear logic and comments which are typical of human-written code. It includes a reasonable number of variables and functions. The style is consistent. ",\n    "answer": "human"\n}\n```',
 '```json\n{\n    "reason": "The code demonstrates a complex algorithm involving string manipulation prefix sums hashing and divisor calculations which is typical of human written code. The code includes detailed comments and a well-structured design with clear variable names and a consistent coding style. The use of a lambda function and the attention to modulo arithmetic further suggests a human author.",\n    "

In [ ]:
answer[0]

'{\n    "answer": "human",\n    "reason": "The code demonstrates a clear understanding of graph traversal algorithms (breadth-first search) and their application to a specific problem. The variable naming is reasonably descriptive, and the code is well-structured with appropriate comments. The use of `ios_base::sync_with_stdio(false);` and `cin.tie(nullptr);` suggests a focus on performance, which is typical in competitive programming. The logic for determining reachability in both forward and backward directions, considering the constraints imposed by the values in the `f` vector (0, 1, and 2), is well-implemented and reflects a human\'s thought process in breaking down the problem."\n}'

In [24]:
import re
import json

result_n = len(answer)
answer_df = test_df.iloc[:result_n]
answer_df['answer_text'] = answer
pattern = re.compile(r'```json\s*(\{.*?\})\s*```', re.DOTALL)
answer_df['answer_dict'] = answer_df['answer_text'].apply(lambda x: x.lstrip('```json\n').rstrip('\n```'))


C:\Users\Pakin\AppData\Local\Temp\ipykernel_25400\446863633.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['answer_text'] = answer
C:\Users\Pakin\AppData\Local\Temp\ipykernel_25400\446863633.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['answer_dict'] = answer_df['answer_text'].apply(lambda x: x.lstrip('```json\n').rstrip('\n```'))


In [25]:
for idx,row in answer_df.iterrows():
    try:
        json.loads(row['answer_dict'])  
    except Exception as e:
        print(f"Error at {idx} :  {e}")
        print(f"Replace with {row['answer_dict'][:50]+row['answer_dict'][-5:]}")
        answer_df['answer_dict'][idx] = row['answer_dict'][:50]+row['answer_dict'][-5:]
        

In [26]:
answer_df['answer_dict'] = answer_df['answer_dict'].apply(lambda x: json.loads(x))
answer_df['model_answer'] = answer_df['answer_dict'].apply(lambda x: x['answer'])
answer_df['model_reason'] = answer_df['answer_dict'].apply(lambda x: x['reason'])
answer_df['verdict'] = answer_df['writer'] == answer_df['model_answer']
answer_df = answer_df[['writer','model_answer','verdict','model_reason','website','source']]

C:\Users\Pakin\AppData\Local\Temp\ipykernel_25400\3813125495.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['answer_dict'] = answer_df['answer_dict'].apply(lambda x: json.loads(x))
C:\Users\Pakin\AppData\Local\Temp\ipykernel_25400\3813125495.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['model_answer'] = answer_df['answer_dict'].apply(lambda x: x['answer'])
C:\Users\Pakin\AppData\Local\Temp\ipykernel_25400\3813125495.py:3: SettingWithCopyWarning: 
A value is trying to be set o

'. "\n}'

In [27]:
filtered_df = pd.DataFrame()
filtered_df[['Label','Model_Answer','Verdict','Reason','Website']] = answer_df[['writer','model_answer','verdict','model_reason','website']]

In [28]:
filtered_df

,Label,Model_Answer,Verdict,Reason,Website
0,Human,human,False,The code demonstrates a clear understanding of...,https://codeforces.com
1,Human,human,False,The code exhibits a structured approach with c...,https://codeforces.com
2,AI,human,False,The code demonstrates a complex algorithm invo...,https://codeforces.com
3,Human,human,False,The code has clear comments uses concise varia...,https://programming.in.th
4,Human,human,False,The code contains unnecessary comments and pre...,https://codeforces.com
...,...,...,...,...,...
2395,Human,human,False,The code includes standard competitive program...,https://codeforces.com
2396,AI,human,False,The code implements a graph algorithm likely t...,https://codeforces.com
2397,AI,human,False,The code demonstrates a well-structured approa...,https://codeforces.com
2398,Human,human,False,The code has a clear structure with nested loo...,https://programming.in.th


In [29]:
# I forgot about upper-lower case T-T
filtered_df['Label'] = filtered_df['Label'].replace('Human', 'human')
filtered_df['Verdict'] = filtered_df['Label']==filtered_df['Model_Answer']

In [30]:
filtered_df['Label'].value_counts(), filtered_df['Model_Answer'].value_counts()

(Label
 human    1200
 AI       1200
 Name: count, dtype: int64,
 Model_Answer
 human    2353
 AI         47
 Name: count, dtype: int64)

In [31]:
filtered_df

,Label,Model_Answer,Verdict,Reason,Website
0,human,human,True,The code demonstrates a clear understanding of...,https://codeforces.com
1,human,human,True,The code exhibits a structured approach with c...,https://codeforces.com
2,AI,human,False,The code demonstrates a complex algorithm invo...,https://codeforces.com
3,human,human,True,The code has clear comments uses concise varia...,https://programming.in.th
4,human,human,True,The code contains unnecessary comments and pre...,https://codeforces.com
...,...,...,...,...,...
2395,human,human,True,The code includes standard competitive program...,https://codeforces.com
2396,AI,human,False,The code implements a graph algorithm likely t...,https://codeforces.com
2397,AI,human,False,The code demonstrates a well-structured approa...,https://codeforces.com
2398,human,human,True,The code has a clear structure with nested loo...,https://programming.in.th


In [32]:
filtered_df.to_csv("./Results/Normal/Result-Gemini2.0.csv",index=False)
# filtered_df.to_csv("./Results/Hard/Result-Gemini2.0.csv",index=False)

In [33]:
accuracy = filtered_df['Verdict'].value_counts(normalize=True).get(True, 0)
print(f"AUC : {accuracy}")

AUC : 0.48875
